In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://nucleus.cels.anl.gov:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /vol/bmd/yanyul/GitHub/ptrs-ukb/notebook/hail-20191223-0944-0.2.28-61941242c15d.log


In [50]:
import pandas as pd

In [142]:
gwas_sample = hl.import_table(
    '/vol/bmd/yanyul/UKB/gwas_on_subset/gwas_runs_in_tsv/gwas_in_tsv_subset13_x_height.tsv',
    key = ['rsid', 'variant']
)

2019-12-23 13:12:33 Hail: INFO: Reading table with no type imputation
  Loading column 'variant' as type 'str' (type not specified)
  Loading column 'minor_allele' as type 'str' (type not specified)
  Loading column 'minor_AF' as type 'str' (type not specified)
  Loading column 'low_confidence_variant' as type 'str' (type not specified)
  Loading column 'n_complete_samples' as type 'str' (type not specified)
  Loading column 'AC' as type 'str' (type not specified)
  Loading column 'ytx' as type 'str' (type not specified)
  Loading column 'beta' as type 'str' (type not specified)
  Loading column 'se' as type 'str' (type not specified)
  Loading column 'tstat' as type 'str' (type not specified)
  Loading column 'pval' as type 'str' (type not specified)
  Loading column 'rsid' as type 'str' (type not specified)
  Loading column 'ref' as type 'str' (type not specified)
  Loading column 'alt' as type 'str' (type not specified)



In [143]:
f = pd.read_csv('../external_data/martin_et_al_2019ng_table_s6_trait_description.tsv', sep = '\t')

In [144]:
traits = [ i.lower() for i in f['short'].to_list() ]

In [145]:
clumped_snp_files = []
for i in traits:
    for j in range(1, 18):
        clumped_snp_files.append('/vol/bmd/yanyul/UKB/ld_clump/gwas_clump_x_subset' + str(j) + '_x_' + i + '.clumped_snp')

In [146]:
len(clumped_snp_files)

289

In [74]:
# !cat /homes/yanyul/gwas_clump_x_subset6_x_sbp.clumped_snp /homes/yanyul/gwas_clump_x_subset6_x_height.clumped_snp|sort|uniq|wc -l

3006244


In [147]:
ht_var = hl.import_table(clumped_snp_files, no_header = True, key = 'f0')

2019-12-23 13:12:53 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)



In [76]:
# ht_var.count()

4015160

In [148]:
ht_var = ht_var.key_by('f0')

In [149]:
gwas_sample = gwas_sample.annotate(is_clump_var = ht_var[gwas_sample.rsid])

In [150]:
ht_var = gwas_sample.filter(hl.is_defined(gwas_sample.is_clump_var))

In [151]:
chr_all = '{' + ','.join([str(i) for i in range(1, 23)]) + '}'
index_dic = {
    f'/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr{i}_v3.bgen' : f'/vol/bmd/yanyul/UKB/bgen_idx/ukb_imp_chr{i}_v3.bgen.idx2' for i in range(1, 23)
}

'{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22}'

In [152]:
mt = hl.import_bgen(
    '/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr' + chr_all + '_v3.bgen', 
    index_file_map = index_dic,
    entry_fields = ['dosage'],
    sample_file = '/vol/bmd/data/ukbiobank/genotypes/v3/ukb19526_imp_chr1_v3_s487395.sample',
    variants = hl.parse_variant(ht_var.variant)
)

2019-12-23 13:13:08 Hail: INFO: Number of BGEN files parsed: 22
2019-12-23 13:13:08 Hail: INFO: Number of samples in BGEN files: 487409
2019-12-23 13:13:08 Hail: INFO: Number of variants across all BGEN files: 93095623
2019-12-23 13:15:26 Hail: INFO: Number of BGEN files parsed: 22
2019-12-23 13:15:26 Hail: INFO: Number of samples in BGEN files: 487409
2019-12-23 13:15:26 Hail: INFO: Number of variants across all BGEN files: 93095623
2019-12-23 13:17:41 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-23 13:18:42 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-23 13:27:23 Hail: INFO: Ordering unsorted dataset with network shuffle


In [153]:
mt.count_rows()

8168109

In [154]:
mt.write('test_output/prs_preprocess.mt')

KeyboardInterrupt: 

In [119]:
indiv_list_files = [ '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/African.txt', '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-1.txt' ]

In [120]:
indiv_list_files = []
for i in range(1, 18):
    indiv_list_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-' + str(i) + '.txt')
    indiv_list_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-validation-' + str(i) + '.txt')
for i in ['African', 'Chinese', 'Indian']:
    indiv_list_files.append('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/' + i + '.txt')

In [121]:
ht_sample = hl.import_table(indiv_list_files, no_header = True, key = 'f0', delimiter = ' ')

2019-12-23 12:13:49 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)
  Loading column 'f1' as type 'str' (type not specified)



In [122]:
ht_sample.show()

f0,f1
str,str
"""1000037""","""1000037"""
"""1000051""","""1000051"""
"""1000084""","""1000084"""
"""1000128""","""1000128"""
"""1000135""","""1000135"""
"""1000144""","""1000144"""
"""1000183""","""1000183"""
"""1000206""","""1000206"""
"""1000319""","""1000319"""


In [124]:
mt = mt.filter_cols(hl.is_defined(ht_sample[mt.s]))

In [ ]:
gwas_run = 

In [ ]:
mt = mt.annotate_rows()